### CELDA 1: Importar librerías 

In [6]:
# CELDA 1: Configuración del Entorno 

import matplotlib
# Le decimos a Matplotlib que use un backend no interactivo compatible con notebooks
matplotlib.use('Agg') 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import folium
from folium.plugins import HeatMap

# Configuramos el estilo de los gráficos para que se vean mejor
sns.set_theme(style="whitegrid")
# Configuramos pandas para que nos muestre más columnas
pd.set_option('display.max_columns', 50)

print("Librerías importadas y backend de Matplotlib configurado a 'Agg'.")

Librerías importadas y backend de Matplotlib configurado a 'Agg'.


In [7]:
# --- INICIO DEL PROCESO DE CARGA Y LIMPIEZA ---

try:
    print("Paso 1: Cargando el archivo 'complaints.csv'...")
    # Carga los datos crudos desde la carpeta data/raw
    df_eda = pd.read_csv('../data/raw/complaints.csv', low_memory=False)
    print(" -> ¡Éxito! Datos cargados.")

    print("\nPaso 2: Estandarizando nombres de columnas a minúsculas...")
    # Solución clave: Estandariza los nombres para evitar errores de mayúsculas/minúsculas
    df_eda.columns = df_eda.columns.str.lower()
    print(" -> ¡Éxito! Nombres estandarizados.")

    print("\nPaso 3: Realizando limpieza básica...")
    # Convierte la columna de fecha. Los errores se convertirán en NaT (Not a Time)
    df_eda['cmplnt_fr_dt'] = pd.to_datetime(df_eda['cmplnt_fr_dt'], errors='coerce')
    
    # Elimina filas con datos cruciales faltantes (fecha, coordenadas, tipo de crimen)
    df_eda.dropna(subset=['cmplnt_fr_dt', 'latitude', 'longitude', 'ofns_desc'], inplace=True)
    print(" -> Filas con valores nulos en columnas clave eliminadas.")
    
    # Filtramos para quedarnos con datos más recientes y manejables
    df_eda = df_eda[df_eda['cmplnt_fr_dt'].dt.year >= 2020].copy()
    print(" -> Filtrado por años >= 2020.")

    print("\n--- ¡PROCESO COMPLETADO! ---")
    print(f"El DataFrame 'df_eda' está limpio y listo para usarse.")
    print(f"Dimensiones finales: {df_eda.shape}")

except FileNotFoundError:
    print("\n!!! ERROR CRÍTICO: No se encontró el archivo 'complaints.csv' en 'data/raw/'.")
    print("    Asegúrate de haber ejecutado el pipeline 'python -m src.flows.main_flow' primero.")
except KeyError as e:
    print(f"\n!!! ERROR CRÍTICO: La columna '{e.args[0]}' no se encontró. Revisa si el nombre ha cambiado en la fuente de datos.")

# Mostramos las primeras 5 filas del DataFrame limpio para verificar
df_eda.head()

Paso 1: Cargando el archivo 'complaints.csv'...
 -> ¡Éxito! Datos cargados.

Paso 2: Estandarizando nombres de columnas a minúsculas...
 -> ¡Éxito! Nombres estandarizados.

Paso 3: Realizando limpieza básica...
 -> Filas con valores nulos en columnas clave eliminadas.
 -> Filtrado por años >= 2020.

--- ¡PROCESO COMPLETADO! ---
El DataFrame 'df_eda' está limpio y listo para usarse.
Dimensiones finales: (998, 35)


,cmplnt_num,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_to_dt,cmplnt_to_tm,addr_pct_cd,rpt_dt,ky_cd,ofns_desc,pd_cd,pd_desc,crm_atpt_cptd_cd,law_cat_cd,boro_nm,loc_of_occur_desc,prem_typ_desc,juris_desc,jurisdiction_code,parks_nm,hadevelopt,housing_psa,x_coord_cd,y_coord_cd,susp_age_group,susp_race,susp_sex,transit_district,latitude,longitude,lat_lon,patrol_boro,station_name,vic_age_group,vic_race,vic_sex
0,298687005,2024-12-30,16:00:00,NaN,(null),42,2024-12-31T00:00:00.000,105,ROBBERY,386.0,"ROBBERY,PERSONAL ELECTRONIC DEVICE",COMPLETED,FELONY,BRONX,(null),STREET,N.Y. POLICE DEPT,0,(null),(null),NaN,1009979,241055,25-44,WHITE HISPANIC,M,NaN,40.828278,-73.907031,"(40.828278257510995, -73.90703069080907)",PATROL BORO BRONX,(null),45-64,WHITE HISPANIC,F
1,298695091,2024-12-31,13:04:00,2024-12-31T00:00:00.000,14:29:00,7,2024-12-31T00:00:00.000,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,COMPLETED,MISDEMEANOR,MANHATTAN,INSIDE,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,2,(null),(null),606.0,989248,198472,25-44,WHITE HISPANIC,M,NaN,40.711446,-73.981970,"(40.711446, -73.98197)",PATROL BORO MAN SOUTH,(null),25-44,BLACK HISPANIC,F
2,298684020,2024-12-31,09:45:00,2024-12-31T00:00:00.000,09:55:00,110,2024-12-31T00:00:00.000,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,COMPLETED,MISDEMEANOR,QUEENS,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0,(null),(null),NaN,1015675,208841,18-24,WHITE HISPANIC,M,NaN,40.739851,-73.886600,"(40.739851, -73.8866)",PATROL BORO QUEENS NORTH,(null),45-64,ASIAN / PACIFIC ISLANDER,M
3,298695155,2024-12-19,16:15:00,2024-12-19T00:00:00.000,16:30:00,48,2024-12-31T00:00:00.000,341,PETIT LARCENY,338.0,"LARCENY,PETIT FROM BUILDING,UN",COMPLETED,MISDEMEANOR,BRONX,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0,(null),(null),NaN,1017623,245367,UNKNOWN,BLACK,F,NaN,40.840099,-73.879387,"(40.840099, -73.879387)",PATROL BORO BRONX,(null),UNKNOWN,UNKNOWN,D
4,298700970,2024-12-31,16:55:00,2024-12-31T00:00:00.000,17:00:00,75,2024-12-31T00:00:00.000,578,HARRASSMENT 2,637.0,"HARASSMENT,SUBD 1,CIVILIAN",COMPLETED,VIOLATION,BROOKLYN,FRONT OF,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0,(null),(null),NaN,1023031,182238,25-44,BLACK,F,NaN,40.666803,-73.860208,"(40.666803, -73.860208)",PATROL BORO BKLYN NORTH,(null),25-44,WHITE,M


In [8]:
print("Análisis de los tipos de crímenes más comunes:")

# Contamos la frecuencia de cada tipo de crimen y seleccionamos el top 15
crime_counts = df_eda['ofns_desc'].value_counts().nlargest(15)

# Creamos la figura para el gráfico
plt.figure(figsize=(12, 8))

# Creamos el gráfico de barras
ax = sns.barplot(x=crime_counts.values, y=crime_counts.index, palette='viridis')

# Añadimos títulos y etiquetas para mayor claridad
ax.set_title('Top 15 Crímenes más Frecuentes en NYC (2020 en adelante)', fontsize=16)
ax.set_xlabel('Número de Incidentes Reportados', fontsize=12)
ax.set_ylabel('Tipo de Crimen', fontsize=12)

plt.show()

Análisis de los tipos de crímenes más comunes:


C:\Users\Diegosuar\AppData\Local\Temp\ipykernel_17580\2433606680.py:10: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(x=crime_counts.values, y=crime_counts.index, palette='viridis')
C:\Users\Diegosuar\AppData\Local\Temp\ipykernel_17580\2433606680.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [9]:
print("Análisis de patrones temporales de crímenes...")

# Creamos las características temporales
df_eda['hour_of_day'] = df_eda['cmplnt_fr_dt'].dt.hour
df_eda['day_of_week'] = df_eda['cmplnt_fr_dt'].dt.day_name()

# Creamos una figura con dos subgráficos
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12))

# Gráfico 1: Crímenes por hora del día
sns.countplot(x='hour_of_day', data=df_eda, ax=ax1, palette='plasma')
ax1.set_title('Distribución de Crímenes por Hora del Día', fontsize=14)
ax1.set_xlabel('Hora (0-23)')
ax1.set_ylabel('Número de Incidentes')

# Gráfico 2: Crímenes por día de la semana
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sns.countplot(x='day_of_week', data=df_eda, ax=ax2, order=days_order, palette='magma')
ax2.set_title('Distribución de Crímenes por Día de la Semana', fontsize=14)
ax2.set_xlabel('Día de la Semana')
ax2.set_ylabel('Número de Incidentes')

# Ajustamos el espaciado y mostramos los gráficos
plt.tight_layout()
plt.show()

Análisis de patrones temporales de crímenes...


C:\Users\Diegosuar\AppData\Local\Temp\ipykernel_17580\1416135341.py:11: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x='hour_of_day', data=df_eda, ax=ax1, palette='plasma')
C:\Users\Diegosuar\AppData\Local\Temp\ipykernel_17580\1416135341.py:18: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x='day_of_week', data=df_eda, ax=ax2, order=days_order, palette='magma')
C:\Users\Diegosuar\AppData\Local\Temp\ipykernel_17580\1416135341.py:25: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [10]:
# CELDA 5: Análisis Geoespacial (CORREGIDA)

import os # Importamos la librería para manejar el sistema de archivos

print("Generando mapa de calor de la densidad de crímenes...")
print("Este proceso puede tardar un poco si hay muchos datos.")

# Tomamos una muestra más pequeña para que el mapa cargue rápido
# Aseguramos que no intentemos muestrear más filas de las disponibles
n_sample = min(20000, len(df_eda))
map_sample = df_eda.sample(n=n_sample, random_state=42)

# Coordenadas del centro de NYC
nyc_center_coords = [40.7128, -74.0060]

# Creamos el mapa base
nyc_heatmap = folium.Map(location=nyc_center_coords, zoom_start=11)

# Preparamos los datos para la capa de calor
heat_data = [[row['latitude'], row['longitude']] for index, row in map_sample.iterrows()]

# Añadimos la capa de calor al mapa
HeatMap(heat_data, radius=12).add_to(nyc_heatmap)

# --- LA SOLUCIÓN ESTÁ AQUÍ ---
# Nos aseguramos de que la carpeta 'reports' exista antes de guardar.
# exist_ok=True evita un error si la carpeta ya fue creada.
os.makedirs('reports', exist_ok=True) 

# Guardamos el mapa como un archivo HTML para el reporte
nyc_heatmap.save('reports/crime_density_heatmap.html')

print("\n¡Éxito! Mapa de calor guardado en 'reports/crime_density_heatmap.html'.")
# Mostramos el mapa en el notebook
nyc_heatmap

Generando mapa de calor de la densidad de crímenes...
Este proceso puede tardar un poco si hay muchos datos.

¡Éxito! Mapa de calor guardado en 'reports/crime_density_heatmap.html'.
